In [1]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import pypulse as pulsar
import matplotlib.pyplot as plt     # para graficar

import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

import os
import shutil

In [11]:
ant = "A2"
year = "2020"
t_min = 2000

In [18]:
# creamos una lista con todos los archivos .pfd de observaciones


os.chdir('./pfds/')                         # entramos al directorio que contiene los .pfds
pfd_files = glob.glob('*' + ant + '_' + year + '*pfd')                           # lista de archivos .pfd

#print(pfd_files)
print("Numero de observaciones = " + str(len(pfd_files)))

#list = open("./results_" + ant + '_' + year + '_subints_' + str(t_min) + '.txt', "w+")        # archivo que contendrá S/N y t_obs
list = open("./results_" + ant + '_' + year + '_subints_' + str(t_min) + '.txt', "a")


# Convert PFD files to PSRFITS

for pfd in pfd_files:
    subprocess.check_call(['psrconv','-o','PSRFITS','-e','fits',pfd])
    
# Save all PSRFITS files
psrfits_files = glob.glob('*' + ant + '_' + year + '*fits')

# Load all PSRFITS into PyPulse
fits_pypulse= []
for psrfits in psrfits_files:
    temp_pypulse= pulsar.Archive(psrfits)
    fits_pypulse.append(temp_pypulse)

observations = dict(zip(psrfits_files, fits_pypulse))

# Create single pulse object
# Creat array of best profiles

single_pulses=[]

for observation in fits_pypulse:
    
    if observation.getDuration() > t_min:
    
    # First crunch in time and frequency
#    print("Duración de la observación total = " + str(observation.getDuration()/60))
#    print("Nbin de la observación total = " + str(observation.getNbin()))
    
        n_subint = int((observation.getDuration()) // t_min)
        print(n_subint)
        
#    print(n_subint)
    
        observation.fscrunch()
        observation.tscrunch(nsubint=n_subint)
    
#    print("Número de subintegraciones = " + str(observation.getNsubint()))
#    print("Duración de la observación total = " + str(observation.getDuration()/60))
#        print("Duración de cada subintegración = " + str(observation.getDurations()/60))
    
    # Now get array of the best profile
        tmp_singlepulse = observation.getSinglePulses( windowsize = int(observation.getNbin()/4 ))
        
        if n_subint == 1:
        
            # Align and normalize
            tmp_singlepulse.center_align()
            tmp_singlepulse.normalize()
        
            file = str(observation).replace(".fits", ".pfd")
            t_obs = observation.getDuration()/60                        # calculamos el tiempo de integración en minutos
            sn_obs = tmp_singlepulse.getSN()                              # calculamos el S/N de la observación
 
            list.write(file + "   " + str(sn_obs) + "   " + str(t_obs) + "\n")  # y escribimos el S/N y el t_obs en el archivo de salida
    
        # Save aligned and normalize in each observation
            single_pulses.append(tmp_singlepulse)
            
        
        elif n_subint > 1:
    
            for n in range(observation.getNsubint()):
            
            # Align and normalize
                tmp_singlepulse[n].center_align()
                tmp_singlepulse[n].normalize()
    
#        print("S/N de la " + str(n) + "-esima subintegración = " + str(tmp_singlepulse[n].getSN()))
        
                file = str(observation).replace(".fits", ".pfd")
                t_obs = observation.getDurations()[n] / 60
                sn_obs = tmp_singlepulse[n].getSN()
 
                list.write(file + "   " + str(sn_obs) + "   " + str(t_obs) + "\n")  # y escribimos el S/N y el t_obs en el archivo de salida
            
                # Save aligned and normalize in each observation
                single_pulses.append(tmp_singlepulse[n])
    
sp_observations = dict(zip(psrfits_files, single_pulses))

list.close()

os.chdir('..')

#shutil.move("./pfds/results_" + ant + '_' + year + '_subints_5000.txt', './')

Numero de observaciones = 8
Loading: prepfold_timing_A2_20200613_105807_PSR_0437-4715.fits
Load time: 5.56 s
Loading: prepfold_timing_A2_20200612_101433_PSR_0437-4715.fits
Load time: 0.85 s
Loading: prepfold_timing_A2_20200601_105803_PSR_0437-4715.fits
Load time: 0.83 s
Loading: prepfold_timing_A2_20200611_122816_PSR_0437-4715.fits
Load time: 1.40 s
Loading: prepfold_timing_A2_20200611_104946_PSR_0437-4715.fits
Load time: 5.06 s
Loading: prepfold_timing_A2_20200612_110457_PSR_0437-4715.fits
Load time: 1.73 s
Loading: prepfold_timing_A2_20200611_134636_PSR_0437-4715.fits
Load time: 2.81 s
Loading: prepfold_timing_A2_20200612_112406_PSR_0437-4715.fits
Load time: 3.48 s
1
1
2
2
3
